# Basic usage of LRP
This example will show you best practices for using LRP,
building on the basics shown in the *Getting started* section.

We start out by loading a small convolutional neural network:

In [1]:
using ExplainableAI
using Flux

model = Chain(
    Chain(
        Conv((3, 3), 3 => 8, relu; pad=1),
        Conv((3, 3), 8 => 8, relu; pad=1),
        MaxPool((2, 2)),
        Conv((3, 3), 8 => 16; pad=1),
        BatchNorm(16, relu),
        Conv((3, 3), 16 => 8, relu; pad=1),
        BatchNorm(8, relu),
    ),
    Chain(
        Flux.flatten,
        Dense(2048 => 512, relu),
        Dropout(0.5),
        Dense(512 => 100, softmax)
    ),
);

This model contains two chains: the convolutional layers and the fully connected layers.

## Model preparation
### Stripping the output softmax
When using LRP, it is recommended to explain output logits instead of probabilities.
This can be done by stripping the output softmax activation from the model
using the `strip_softmax` function:

In [2]:
model = strip_softmax(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

If you don't remove the output softmax,
model checks will fail.

### Canonizing the model
LRP is not invariant to a model's implementation.
Applying the `GammaRule` to two linear layers in a row will yield different results
than first fusing the two layers into one linear layer and then applying the rule.
This fusing is called "canonization" and can be done using the `canonize` function:

In [3]:
model = canonize(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

### Flattening the model
ExplainableAI.jl's LRP implementation supports nested Flux Chains and Parallel layers.
However, it is recommended to flatten the model before analyzing it.

LRP is implemented by first running a forward pass through the model,
keeping track of the intermediate activations, followed by a backward pass
that computes the relevances.

To keep the LRP implementation simple and maintainable,
ExplainableAI.jl does not pre-compute "nested" activations.
Instead, for every internal chain, a new forward pass is run to compute activations.

By "flattening" a model, this overhead can be avoided.
For this purpose, ExplainableAI.jl provides the function `flatten_model`:

In [4]:
model_flat = flatten_model(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

This function is called by default when creating an LRP analyzer.
Note that we pass the unflattened model to the analyzer, but `analyzer.model` is flattened:

In [5]:
analyzer = LRP(model)
analyzer.model

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

If this flattening is not desired, it can be disabled
by passing the keyword argument `flatten=false` to the `LRP` constructor.

## LRP rules
By default, the `LRP` constructor will assign the `ZeroRule` to all layers.

In [6]:
LRP(model)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => ZeroRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZeroRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, relu, pad=1) => ZeroRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZeroRule(),
  BatchNorm(8, relu)                 => ZeroRule(),
  Flux.flatten                       => ZeroRule(),
  Dense(2048 => 512, relu)           => ZeroRule(),
  Dropout(0.5)                       => ZeroRule(),
  Dense(512 => 100)                  => ZeroRule(),
)

This analyzer will return heatmaps that look identical to `InputTimesGradient`.

LRP's strength lies in assigning different rules to different layers,
based on their functionality in the neural network[^1].
ExplainableAI.jl implements many LRP rules out of the box,
but it is also possible to *implement custom rules*.

To assign different rules to different layers,
use one of the composites presets,
or create your own composite, as described in
*Assigning rules to layers*.

In [7]:
composite = EpsilonPlusFlat() # using composite preset EpsilonPlusFlat

Composite(
  GlobalTypeMap(  # all layers
    Flux.Conv               => ZPlusRule(),
    Flux.ConvTranspose      => ZPlusRule(),
    Flux.CrossCor           => ZPlusRule(),
    Flux.Dense              => EpsilonRule{Float32}(1.0f-6),
    typeof(NNlib.dropout)   => PassRule(),
    Flux.AlphaDropout       => PassRule(),
    Flux.Dropout            => PassRule(),
    Flux.BatchNorm          => PassRule(),
    typeof(Flux.flatten)    => PassRule(),
    typeof(MLUtils.flatten) => PassRule(),
    typeof(identity)        => PassRule(),
 ),
  FirstLayerTypeMap(  # first layer
    Flux.Conv          => FlatRule(),
    Flux.ConvTranspose => FlatRule(),
    Flux.CrossCor      => FlatRule(),
    Flux.Dense         => FlatRule(),
 ),
)

In [8]:
LRP(model, composite)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => FlatRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZPlusRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, relu, pad=1) => ZPlusRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZPlusRule(),
  BatchNorm(8, relu)                 => PassRule(),
  Flux.flatten                       => PassRule(),
  Dense(2048 => 512, relu)           => EpsilonRule{Float32}(1.0f-6),
  Dropout(0.5)                       => PassRule(),
  Dense(512 => 100)                  => EpsilonRule{Float32}(1.0f-6),
)

## Computing layerwise relevances
If you are interested in computing layerwise relevances,
call `analyze` with an LRP analyzer and the keyword argument
`layerwise_relevances=true`.

The layerwise relevances can be accessed in the `extras` field
of the returned `Explanation`:

In [9]:
input = rand(Float32, 32, 32, 3, 1) # dummy input for our convolutional neural network

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

11-element Vector{Array{Float32}}:
 [0.11555797 0.16567013 … -0.00074588886 -0.016081242; -0.020463338 -0.18181762 … 0.0075527276 -0.0012225641; … ; 0.05978778 0.025775693 … 0.016010145 0.0058790017; 0.027305666 0.034312364 … 0.011098867 0.0031443986;;; 0.17919934 -0.44290525 … -0.0028781097 0.0025866567; -0.17921384 -1.1027005 … 0.00529778 0.014584399; … ; 0.0065630525 0.019256484 … 0.01691892 -0.017772272; -0.006193216 -0.0006287951 … -0.0018861903 0.0041949633;;; -0.23348749 -0.14968054 … -0.017748682 0.005836729; 0.22589077 -0.11296646 … 0.0004931396 0.008539669; … ; 0.0057462584 -0.0026442981 … -0.015646847 -0.0002237301; 0.0056847427 -0.013524553 … 0.01720984 -0.004970015;;;;]
 [-0.037589494 -0.35138637 … -0.0023543204 -0.0031317736; -0.32994968 -0.343098 … -0.0023740756 -0.001340456; … ; 0.001353058 -0.045696773 … 0.024047514 -0.0043757693; 0.009031263 0.00071556837 … -0.012437812 -0.0006313002;;; 0.12511042 0.16571468 … 0.0062297885 -0.0023807054; 0.032985922 0.043249875 … 0.00

Note that the layerwise relevances are only kept for layers in the outermost `Chain` of the model.
When using our unflattened model, we only obtain three layerwise relevances,
one for each chain in the model and the output relevance:

In [10]:
analyzer = LRP(model; flatten=false) # use unflattened model

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

11-element Vector{Array{Float32}}:
 [0.10909061 0.016985286 … -0.033392604 0.035766583; 0.0047575007 -0.10714021 … -0.012110931 0.0013750129; … ; -0.07479966 0.007444864 … 0.036861412 0.009906101; 0.00805714 -0.022324992 … 0.01221438 0.045526262;;; 0.1833335 -0.31012642 … 0.0042630946 -0.015650481; -0.101551116 -0.8807603 … 0.00069726567 -0.030251766; … ; -0.015744127 -0.03496587 … -0.015696604 -0.014254219; -0.023524927 -0.024451341 … -0.002061537 0.015031568;;; -0.12570041 -0.118935265 … 0.015687658 0.011928017; 0.14294061 -0.07356083 … 0.01841417 0.04019364; … ; 0.025564123 -0.013152767 … -0.009696721 -0.0025443526; -0.003473982 0.010800932 … 0.017462572 0.0029346414;;;;]
 [-0.036506597 -0.25065005 … 0.01619241 0.011507603; -0.22996363 -0.35731292 … -0.004716799 0.0038788142; … ; -0.033761796 0.0313675 … 0.022528403 -0.0051091155; -0.0072388975 -0.008173408 … 0.009247528 -0.0056583444;;; 0.08575319 0.17711058 … -0.007875867 0.00058436993; 0.013789675 0.09537185 … -0.0010806506 -0.00

## Performance tips
### Using LRP without a GPU
Since ExplainableAI.jl's LRP implementation makes use of
[Tullio.jl](https://github.com/mcabbott/Tullio.jl),
analysis can be accelerated by loading either
- a package from the [JuliaGPU](https://juliagpu.org) ecosystem,
  e.g. [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl), if a GPU is available
- [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
  if only a CPU is available.

This only requires loading the LoopVectorization.jl package before ExplainableAI.jl:
```julia
using LoopVectorization
using ExplainableAI
```

[^1]: G. Montavon et al., [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007/978-3-030-28954-6_10)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*